# Lab 3.3.3: TensorRT-LLM Optimization - Solutions

This notebook provides solutions for the exercises in the TensorRT-LLM optimization notebook.

## Exercise 1: Build a TensorRT Engine

Document your build process for Llama 3.1 8B.

In [ ]:
# Solution: TensorRT-LLM Build Process Documentation

build_config = {
    "model": "meta-llama/Llama-3.1-8B-Instruct",
    "dtype": "bfloat16",
    "max_input_len": 4096,
    "max_output_len": 2048,
    "max_batch_size": 8,
    "plugins": ["gemm_plugin", "use_fused_mlp"],
    "paged_kv_cache": True,
}

# Step 1: Start the container
container_cmd = """
docker run --gpus all -it --rm \\
    -v ~/trtllm-workspace:/workspace \\
    -v ~/.cache/huggingface:/root/.cache/huggingface \\
    -e HF_TOKEN=$HF_TOKEN \\
    --ipc=host \\
    --shm-size=16g \\
    --ulimit memlock=-1 \\
    nvcr.io/nvidia/tritonserver:25.11-trtllm-python-py3 \\
    bash
"""

# Step 2: Convert checkpoint (inside container)
convert_cmd = """
python /opt/TensorRT-LLM/examples/llama/convert_checkpoint.py \\
    --model_dir meta-llama/Llama-3.1-8B-Instruct \\
    --output_dir /workspace/checkpoints/llama-8b \\
    --dtype bfloat16
"""

# Step 3: Build engine (inside container)
build_cmd = """
trtllm-build \\
    --checkpoint_dir /workspace/checkpoints/llama-8b \\
    --output_dir /workspace/engines/llama-8b \\
    --max_input_len 4096 \\
    --max_seq_len 6144 \\
    --max_batch_size 8 \\
    --gemm_plugin bfloat16 \\
    --use_fused_mlp enable \\
    --paged_kv_cache enable
"""

print("TensorRT-LLM Build Configuration:")
print("=" * 50)
for key, value in build_config.items():
    print(f"  {key}: {value}")

print("\nCommands:")
print("1. Container:", container_cmd.strip()[:60] + "...")
print("2. Convert:", convert_cmd.strip()[:60] + "...")
print("3. Build:", build_cmd.strip()[:60] + "...")

In [ ]:
# Results template - fill in after building

build_results = {
    "build_time_minutes": 55,  # Example: Llama 8B on DGX Spark
    "engine_size_gb": 15.2,    # Approximate for bfloat16
    "checkpoint_size_gb": 16.0,
    "errors_encountered": [],   # Any issues during build
    "notes": [
        "Build completed successfully",
        "Used --shm-size=16g to avoid memory issues",
        "Engine is ~5% smaller than raw checkpoint due to optimization"
    ]
}

print("Build Results:")
print("=" * 50)
print(f"Build Time: {build_results['build_time_minutes']} minutes")
print(f"Engine Size: {build_results['engine_size_gb']} GB")
print(f"Checkpoint Size: {build_results['checkpoint_size_gb']} GB")
print("\nNotes:")
for note in build_results['notes']:
    print(f"  - {note}")

## Exercise 2: Prefill Scaling Test

Test how TensorRT-LLM's prefill speed scales with input length.

In [ ]:
# Solution: Prefill Scaling Test

def create_prompt_of_length(target_tokens: int) -> str:
    """
    Create a prompt with approximately target_tokens tokens.
    Using repeated text for consistent behavior.
    """
    base_text = "The quick brown fox jumps over the lazy dog. "
    words_per_token = 0.75  # Approximate for Llama tokenizer
    words_needed = int(target_tokens * words_per_token)
    
    # Repeat base text
    repeated = (base_text * (words_needed // 9 + 1))[:words_needed * 5]
    
    prompt = f"Summarize the following text in one sentence:\n\n{repeated}\n\nSummary:"
    return prompt

# Create prompts of different lengths
test_lengths = [100, 500, 1000, 2000, 4000]
test_prompts = {length: create_prompt_of_length(length) for length in test_lengths}

print("Prompt lengths created:")
for length, prompt in test_prompts.items():
    actual_words = len(prompt.split())
    print(f"  {length} tokens target -> {actual_words} words (~{int(actual_words/0.75)} tokens)")

In [ ]:
# Benchmark prefill at different lengths
import time
import requests
import json

TRTLLM_URL = "http://localhost:8000"

def benchmark_prefill(prompt: str, max_tokens: int = 50) -> dict:
    """
    Measure prefill time (time to first token).
    """
    start = time.perf_counter()
    first_token_time = None
    
    try:
        response = requests.post(
            f"{TRTLLM_URL}/v1/chat/completions",
            json={
                "model": "tensorrt_llm",
                "messages": [{"role": "user", "content": prompt}],
                "max_tokens": max_tokens,
                "stream": True
            },
            stream=True,
            timeout=120
        )
        
        for line in response.iter_lines():
            if line:
                line_str = line.decode()
                if line_str.startswith("data: ") and "[DONE]" not in line_str:
                    try:
                        chunk = json.loads(line_str[6:])
                        if chunk.get("choices", [{}])[0].get("delta", {}).get("content"):
                            if first_token_time is None:
                                first_token_time = time.perf_counter()
                    except:
                        pass
        
        end = time.perf_counter()
        prefill_time = (first_token_time - start) if first_token_time else None
        
        return {
            "prefill_time_ms": prefill_time * 1000 if prefill_time else None,
            "total_time_ms": (end - start) * 1000
        }
    except Exception as e:
        return {"error": str(e)}

# Expected results (simulated for demonstration)
expected_results = {
    100: {"prefill_time_ms": 15, "prefill_tokens_per_sec": 6667},
    500: {"prefill_time_ms": 42, "prefill_tokens_per_sec": 11905},
    1000: {"prefill_time_ms": 75, "prefill_tokens_per_sec": 13333},
    2000: {"prefill_time_ms": 140, "prefill_tokens_per_sec": 14286},
    4000: {"prefill_time_ms": 270, "prefill_tokens_per_sec": 14815},
}

print("Expected Prefill Scaling Results (TensorRT-LLM):")
print("=" * 60)
print(f"{'Tokens':<10} {'Prefill Time (ms)':<20} {'Speed (tok/s)':<15}")
print("-" * 50)
for tokens, result in expected_results.items():
    print(f"{tokens:<10} {result['prefill_time_ms']:<20} {result['prefill_tokens_per_sec']:<15}")

print("\nKey Insight: TRT-LLM prefill speed INCREASES with longer inputs!")
print("This is because the GPU can process more tokens in parallel.")

## Key Takeaways

1. **Build time** is significant (~45-90 min for 8B) but engine runs much faster
2. **Prefill scales well** - longer inputs get higher tokens/second
3. **TRT-LLM is ideal** for RAG and long-context applications